In [1]:
from bokeh.plotting import figure, save, output_file, show
from bokeh.models import (ColumnDataSource, DataTable, DateFormatter, 
                          TableColumn, HoverTool, PolyAnnotation, CategoricalColorMapper,
                         Circle)
from bokeh.transform import factor_cmap
from bokeh.layouts import row, column
from bokeh.io import output_notebook
from bokeh.embed import file_html
from bokeh.resources import CDN
import pandas as pd
from datetime import date, timedelta


output_notebook()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


Loading BokehJS ...

In [2]:
run_dt = str(date.today() + timedelta(days=-1))

In [3]:
df = pd.read_csv("data/tto_events.csv")

plot_data = df["events"].value_counts()
outcomes = list(plot_data.index)
# print(plot_data.columns)



In [4]:
source = ColumnDataSource(df)

columns = [
        TableColumn(field="batter", title="Batter"),
        TableColumn(field="pitch_type", title="Pitch Type"),
        TableColumn(field="release_speed", title="Pitch Speed"),
        TableColumn(field="events", title="Outcome"),
        TableColumn(field="launch_speed", title="Exit Velo"),
        TableColumn(field="plate_x", title="Plate X"),
        TableColumn(field="plate_z", title="Plate Z"),
    ]


In [5]:
data_table = DataTable(source=source, 
                       columns=columns, 
                       width=800, 
                       height=280)



In [17]:
group = df.groupby("events")
group

In [18]:
group.describe()

release_speed                                                   \
                  count       mean       std   min    25%    50%     75%   
events                                                                     
home_run           32.0  87.981250  5.834460  75.1  85.95  88.40  92.600   
strikeout         270.0  88.487407  5.723576  78.0  83.40  87.85  93.550   
walk               86.0  89.916279  4.768990  78.0  86.65  90.30  93.825   

                 release_pos_x            ... delta_home_win_exp         \
             max         count      mean  ...                75%    max   
events                                    ...                             
home_run    98.2          32.0 -1.092188  ...            0.14675  0.381   
strikeout  102.3         270.0 -0.855630  ...            0.02000  0.123   
walk       100.8          86.0 -0.968605  ...            0.02575  0.120   

          delta_run_exp                                                      \
                  count      mean       std    min     25%     50%      75%   
events                                                                        
home_run           32.0  1.564500  0.700721  0.955  0.9965  1.5535  1.86475   
strikeout         270.0 -0.219248  0.128908 -0.911 -0.2760 -0.1995 -0.15000   
walk               86.0  0.235000  0.110317  0.038  0.1660  0.2400  0.31300   

                  
             max  
events            
home_run   3.539  
strikeout  0.446  
walk       0.621  

[3 rows x 600 columns]

In [30]:
source.selected.indices

[]

In [38]:
# create a new plot with a title and axis labels
group = df.groupby("events")

p = figure(title="Count of All Events for {}".format(run_dt),
            x_range=group,
            x_axis_label='Outcomes',
            y_axis_label='Frequency',
            width=400, 
            height=400)

# add a line renderer with legend and line thickness to the plot

outcome_counts = p.vbar(x="events", 
                        bottom=0, 
                        top="strikes_count",
                        source=group,
                        width=0.8)

tooltips = [
    ("Freqency ", "@events: @strikes_count"),
]

outcome_hover = HoverTool(renderers=[outcome_counts], tooltips=tooltips)
p.add_tools(outcome_hover)


show(p)

In [7]:
factors = df["events"].unique()

In [8]:
len(factors)

3

In [9]:
# make a strike zone figure

sz = figure(width=400,
            height=400, 
            tools="box_select,reset", 
            active_drag="box_select")

# I think this plot from the center
sz.rect(0, 2.5, 2*0.71, 2,
        fill_alpha = 0,
        line_width = 3,
        line_color = "gray"
       )

home_plate_coords = [[-0.71, 0], [-0.85, -0.5], [0, -1], [0.85, -0.5], [0.71, 0]]

home_plate = PolyAnnotation(
    fill_color="lightgray",
    fill_alpha=0.5,
    line_color="darkgray",
    line_width=3,
    xs=list(map(lambda coord: coord[0], home_plate_coords)),
    ys=list(map(lambda coord: coord[1], home_plate_coords)),
)

sz.add_layout(home_plate)

pitches = sz.circle(x="plate_x",
                    y="plate_z", 
                    color=factor_cmap('events',
                                      palette="Spectral4",
                                      factors=factors),
                    size=25,
                    alpha=0.4,
                    source=source)

# change the styling for selected v not
selected_circle = Circle(fill_alpha=0.6, line_color = None, fill_color=factor_cmap('events',
                                      palette="Spectral4",
                                      factors=factors))
nonselected_circle = Circle(fill_alpha=0.1, line_color = None, fill_color=factor_cmap('events',
                                      palette="Spectral4",
                                      factors=factors),)

pitches.selection_glyph = selected_circle
pitches.nonselection_glyph = nonselected_circle


tooltips = [
    ("Batter", "@batter"),
    ("Pitch Type", "@pitch_type"),
    ("Pitch Speed", "@release_speed"),
    ("Outcome", "@events"),
    ("Exit Velo", "@launch_speed"),
    ("Plate X", "@plate_x"),
    ("Plate Z", "@plate_z")
]

pitches_hover_tool = HoverTool(renderers=[pitches], tooltips=tooltips)

sz.add_tools(pitches_hover_tool)


sz.axis.visible = False
sz.grid.visible = False

In [10]:
show(column(data_table, row(p, sz)))

with open("index_template.html") as template:
    template = template.read()

out_html = file_html(column(data_table, row(p, sz)),
                     CDN,
                     template=template,
                     template_variables={"run_dt" : run_dt},
                     title="Three True Outcomes for {}".format(run_dt))

with open("index.html", "w") as output:
    output.write(out_html)


output_file('index.html',
       ?     template="index_template.html",
            title="Three True Outcomes for {}".format(run_dt))

save(row(p, data_table))